<a href="https://colab.research.google.com/github/vale314/Segmentention-Brain/blob/test/unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocesado

In [ ]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
plt.set_cmap('gray')
import cv2
import glob

<Figure size 432x288 with 0 Axes>

In [ ]:
!pip install pynrrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 23.8 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


In [ ]:
import nrrd

In [ ]:
DATA_INPUT_PATH = '/content/drive/MyDrive/IAIM/project/volumes/'
IMG_INPUT_PATH = os.path.join(DATA_INPUT_PATH, 'img/')
MASK_INPUT_PATH = os.path.join(DATA_INPUT_PATH, 'mask/')

DATA_OUTPUT_PATH = '/content/drive/MyDrive/IAIM/project/slices/'
IMG_SLICE_OUTPUT_PATH = os.path.join(DATA_OUTPUT_PATH, 'img/')
MASK_SLICE_OUTPUT_PATH = os.path.join(DATA_OUTPUT_PATH, 'mask/')

In [ ]:
img_path = os.path.join(IMG_INPUT_PATH, 'sub-001_T1w.nii.gz')
img = nib.load(img_path).get_fdata()
img.min(), img.max(), img.shape, type(img)

FileNotFoundError: ignored

In [ ]:
img_slice = img[90,:,:]
plt.imshow(img_slice)

In [ ]:
mask_path = os.path.join(MASK_INPUT_PATH, 'sub-001_T1w_mask.nrrd')
mask = nrrd.read(mask_path)[0]
mask.min(), mask.max(), mask.shape, type(mask)

In [ ]:
mask_slice = mask[90,:,:]
plt.imshow(mask_slice)

In [ ]:
def normalize_image_intensity(img):
    lmin = img.min()
    lmax = img.max()
    return (img-lmin) / (lmax-lmin)
n_img = normalize_image_intensity(img)
n_img.min(), n_img.max(), n_img.shape, type(n_img)

In [ ]:
def read_image_volume(path, normalize=False):
    try:
        img = nib.load(path).get_fdata()
    except:
        print('No es un archivo .nii.gz, intentando abrir .nrrd...')
        img = nrrd.read(path)[0]
    if normalize:
        return normalize_image_intensity(img)
    return img
read_image_volume(img_path, normalize=True)
read_image_volume(mask_path, normalize=False)

In [ ]:
def save_slice(img, fname, path):
    img = np.uint(img * 255)
    fout = os.path.join(path, f'{fname}.png')
    cv2.imwrite(fout, img)
    print(f'\rCorte guardado: {fout}', end='')
#save_slice(n_img[90,:,:], 'test', IMG_SLICE_OUTPUT_PATH)
#save_slice(mask[90,:,:], 'test', MASK_SLICE_OUTPUT_PATH)

In [ ]:
def slice_and_save_volume(vol, fname, path):
    y, x, z = vol.shape
    print('\nCortes en Y...')
    for i in range(y):
        save_slice(vol[i,:,:], fname+f'-slice{str(i).zfill(3)}_y', path)
    print('\nCortes en X...')
    for i in range(x):
        save_slice(vol[:,i,:], fname+f'-slice{str(i).zfill(3)}_x', path)
    print('\nCortes en Z...')
    for i in range(z):
        save_slice(vol[:,:,i], fname+f'-slice{str(i).zfill(3)}_z', path)
    return y+x+z

In [ ]:
# Se crean las carpetas en colab
!mkdir -p slices/{test/{img/img,mask/img},training/{img/img,mask/img}}

In [ ]:
from skimage.transform import resize
total_img_slices = 0
for index, filename in enumerate(sorted(glob.iglob(IMG_INPUT_PATH+'*.nii.gz'))[:14]):
    img = read_image_volume(filename, True)
    img = resize(img, (180, 240, 240), order=0)
    print(filename, img.shape, np.sum(img.shape), img.min(), img.max())
    n_slices = slice_and_save_volume(img, 'brain'+str(index), '/content/slices/training/img/img')
    print(f'\n{filename}, {n_slices} cortes creados \n')
    total_img_slices += n_slices
print(f'Creados {total_img_slices} en total')

Creados 0 en total


In [ ]:
total_img_slices = 0
for index, filename in enumerate(sorted(glob.iglob(IMG_INPUT_PATH+'*.nii.gz'))[14:]):
    img = read_image_volume(filename, True)
    img = resize(img, (180, 240, 240), order=0)
    print(filename, img.shape, np.sum(img.shape), img.min(), img.max())
    n_slices = slice_and_save_volume(img, 'brain'+str(index), '/content/slices/test/img/img')
    print(f'\n{filename}, {n_slices} cortes creados \n')
    total_img_slices += n_slices
print(f'Creados {total_img_slices} en total')

In [ ]:
# total_img_slices = 0
# for index, filename in enumerate(sorted(glob.iglob(IMG_INPUT_PATH+'*.nii.gz'))):
#     img = read_image_volume(filename, True)
#     print(filename, img.shape, np.sum(img.shape), img.min(), img.max())
#     n_slices = slice_and_save_volume(img, 'brain'+str(index), IMG_SLICE_OUTPUT_PATH)
#     print(f'\n{filename}, {n_slices} cortes creados \n')
#     total_img_slices += n_slices
# print(f'Creados {total_img_slices} en total')

In [ ]:
total_mask_slices = 0
for index, filename in enumerate(sorted(glob.iglob(MASK_INPUT_PATH+'*.nrrd'))[:14]):
    img = read_image_volume(filename, False)
    img = resize(img, (180, 240, 240), order=0)
    print(filename, img.shape, np.sum(img.shape), img.min(), img.max())
    n_slices = slice_and_save_volume(img, 'brain'+str(index), '/content/slices/training/mask/img')
    print(f'\n{filename}, {n_slices} cortes creados \n')
    total_mask_slices += n_slices
print(f'Creados {total_mask_slices} en total')

In [ ]:
total_mask_slices = 0
for index, filename in enumerate(sorted(glob.iglob(MASK_INPUT_PATH+'*.nrrd'))[14:]):
    img = read_image_volume(filename, False)
    img = resize(img, (180, 240, 240), order=0)
    print(filename, img.shape, np.sum(img.shape), img.min(), img.max())
    n_slices = slice_and_save_volume(img, 'brain'+str(index), '/content/slices/test/mask/img')
    print(f'\n{filename}, {n_slices} cortes creados \n')
    total_mask_slices += n_slices
print(f'Creados {total_mask_slices} en total')

In [ ]:
# total_mask_slices = 0
# for index, filename in enumerate(sorted(glob.iglob(MASK_INPUT_PATH+'*.nrrd'))):
#     img = read_image_volume(filename, False)
#     print(filename, img.shape, np.sum(img.shape), img.min(), img.max())
#     n_slices = slice_and_save_volume(img, 'brain'+str(index), MASK_SLICE_OUTPUT_PATH)
#     print(f'\n{filename}, {n_slices} cortes creados \n')
#     total_mask_slices += n_slices
# print(f'Creados {total_mask_slices} en total')

# Entrenamiento

In [ ]:
import os
import matplotlib.pyplot as plt
plt.set_cmap('gray')

import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from tensorflow import keras

In [ ]:
SEED = 909
BATCH_SIZE_TRAIN = 32
BATCH_SIZE_TEST = 32

IMG_HEIGHT = 120
IMG_WIDTH = 120
IMG_SIZE = IMG_HEIGHT, IMG_WIDTH

DATA_DIR = '/content/slices/'

DATA_DIR_TRAIN = os.path.join(DATA_DIR, 'training')
DATA_DIR_TRAIN_IMG = os.path.join(DATA_DIR_TRAIN, 'img')
DATA_DIR_TRAIN_MASK = os.path.join(DATA_DIR_TRAIN, 'mask')

DATA_DIR_TEST = os.path.join(DATA_DIR, 'test')
DATA_DIR_TEST_IMG = os.path.join(DATA_DIR_TEST, 'img')
DATA_DIR_TEST_MASK = os.path.join(DATA_DIR_TEST, 'mask')

In [ ]:
def create_segmentation_generator_train(img_path, mask_path, BATCH_SIZE):
    data_gen_args = dict(rescale=1./255)
    
    datagen = ImageDataGenerator(**data_gen_args)

    img_generator = datagen.flow_from_directory(
        img_path, target_size=IMG_SIZE, class_mode=None, 
        color_mode='grayscale', batch_size=BATCH_SIZE, 
        seed=SEED
        )
    
    mask_generator = datagen.flow_from_directory(
        mask_path, target_size=IMG_SIZE, class_mode=None, 
        color_mode='grayscale', batch_size=BATCH_SIZE, 
        seed=SEED
        )

    return zip(img_generator, mask_generator)

In [ ]:
train_generator = create_segmentation_generator_train(DATA_DIR_TRAIN_IMG, DATA_DIR_TRAIN_MASK, BATCH_SIZE_TRAIN)
test_generator = create_segmentation_generator_train(DATA_DIR_TEST_IMG, DATA_DIR_TEST_MASK, BATCH_SIZE_TEST)

In [ ]:
def display(*display_list):
    plt.figure(figsize=(15,15))
    
    title = ['Imagen de entrada', 'Máscara verdadera', 'Predicción']
    
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]), cmap='gray')
    plt.show()

In [ ]:
def show_dataset(datagen, num=1):
    for i in range(0,num):
        image, mask = next(datagen)
        display(image[0], mask[0])

In [ ]:
show_dataset(train_generator, 2)

In [ ]:
def unet(n_levels, initial_features=32, n_blocks=2, kernel_size=3, pooling_size=2, in_channels=1, out_channels=1):
    inputs = keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, in_channels))
    x = inputs
    
    convpars = dict(kernel_size=kernel_size, activation='relu', padding='same')
    
    # downstream
    skips = {}
    for level in range(n_levels):
        for _ in range(n_blocks):
            x = keras.layers.Conv2D(initial_features * 2 ** level, **convpars)(x)
        if level < n_levels - 1:
            skips[level] = x
            x = keras.layers.MaxPool2D(pooling_size)(x)
            
    # upstream
    for level in reversed(range(n_levels-1)):
        x = keras.layers.Conv2DTranspose(initial_features * 2 ** level, strides=pooling_size, **convpars)(x)
        x = keras.layers.Concatenate()([x, skips[level]])
        for _ in range(n_blocks):
            x = keras.layers.Conv2D(initial_features * 2 ** level, **convpars)(x)
            
    # output
    activation = 'sigmoid' if out_channels == 1 else 'softmax'
    x = keras.layers.Conv2D(out_channels, kernel_size=1, activation=activation, padding='same')(x)
    
    return keras.Model(inputs=[inputs], outputs=[x], name=f'UNET-L{n_levels}-F{initial_features}')

NUM_TRAIN = 9240
NUM_TEST = 3960

EPOCH_STEP_TRAIN = NUM_TRAIN // BATCH_SIZE_TRAIN
EPOCH_STEP_TEST = NUM_TEST // BATCH_SIZE_TEST

model = unet(4)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
NUM_OF_EPOCHS = 100

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=EPOCH_STEP_TRAIN,
    validation_data=test_generator,
    validation_steps=EPOCH_STEP_TEST,
    epochs=NUM_OF_EPOCHS
    )

In [ ]:
model.save(f'/content/drive/MyDrive/IAIM/project/UNET-BrainSegmentation_{IMG_HEIGHT}_{IMG_WIDTH}.h5')

In [ ]:
def create_segmentation_generator_test(img_path, msk_path, BATCH_SIZE):
    data_gen_args = dict(rescale=1./255)
    datagen = ImageDataGenerator(**data_gen_args)
    
    img_generator = datagen.flow_from_directory(
        img_path,
        target_size=IMG_SIZE,
        class_mode=None,
        color_mode='grayscale',
        batch_size=BATCH_SIZE, seed=SEED
    )
    msk_generator = datagen.flow_from_directory(msk_path, target_size=IMG_SIZE, class_mode=None, color_mode='grayscale', batch_size=BATCH_SIZE, seed=SEED)
    return zip(img_generator, msk_generator)

In [ ]:
test_generator = create_segmentation_generator_test(DATA_DIR_TEST_IMG, DATA_DIR_TEST_MASK, 1)

In [ ]:

def show_prediction(datagen, num=1):
    for i in range(0,num):
        image, mask = next(datagen)
        pred_mask = model.predict(image)[0]
        print(pred_mask.max())
        display (image[0], mask[0], pred_mask)

In [ ]:
show_prediction(test_generator, 3)